In [1]:
from fastai import *
from fastai.vision import *


# https://towardsdatascience.com/autoencoders-in-the-fastai-library-fa288e1f899a# 

In [2]:
class ImageSiTuple(ItemBase):
    def __init__(self, img, si_img):
        self.img,self.si_img = img,si_img
        self.obj = (img,si_img),
        self._update_data()
        
    def _update_data(self):
        #self.data =[-1+2*self.img.data,-1+2*self.si_img.data]
        self.data =[self.img.data,self.si_img.data]
        
    def apply_tfms(self, tfms, **kwargs):
        self.img = self.img.apply_tfms(tfms, **kwargs)
        self.si_img = self.si_img.apply_tfms(tfms, **kwargs)
        self._update_data()
        return self
    
    def to_one(self): 
        #return Image(0.5+torch.cat(self.data,2)/2)
        return Image(torch.cat(self.data,2))

In [3]:
class SegmentationProcessor(PreProcessor):
    def __init__(self, ds:ItemList): self.classes = ds.classes
    def process(self, ds:ItemList):  ds.classes,ds.c = self.classes,len(self.classes)

In [4]:
class TargetSiList(ItemList):
    def reconstruct(self, t:Tensor): 
        if len(t.size()) == 0: return t
        return ImageTuple(Image(t[0]/2+0.5),Image(t[1]/2+0.5))

In [19]:
class SideinformationImageImageList(ImageList):
    _label_cls=TargetSiList
    def __init__(self, items, si_items=None, **kwargs):
        super().__init__(items, **kwargs)
        self.si_items = si_items
        self.copy_new.append('si_items')
        
    def get(self, i):
        # get an image, opened by self.open()
        img = super().get(i) 
        si_img_file_name = self.si_items[i]
        return ImageTuple(img=img, si_img=self.open(si_img_file_name))
  
    def open(self, fn):
        "Open image in `fn`, subclass and overwrite for custom behavior."
        return open_image(fn, convert_mode=self.convert_mode, after_open=self.after_open)
    
    def reconstruct(self, x):
        #def reconstruct(self, t, x):
        """Called in data.show_batch(), learn.predict() or learn.show_results()
        Transform tensor back into an ItemBase
        
        DONT CHANGE x !
        """
        return ImageSiTuple(Image(t[0]), Image(t[1]))

    @classmethod
    def from_csv(cls, path:PathOrStr, csv_name:str, header:str=None, delimiter:str='\n', **kwargs)->'SideinformationImageImageList':
        "Get the filenames in `path/csv_name` opened with `header`."
        path = Path(path)
        df = pd.read_csv(path/csv_name, header=None, delimiter=delimiter)
        
        # image path's are intermittent
        img_df, si_img_df=df.iloc[1::2], df.iloc[0::2]
              
        si_items = ImageList.from_df(path=path, df=si_img_df,**kwargs).items
        res = super().from_df(path=path, df=img_df,si_items=si_items, **kwargs)
        return res
    
    def show_xys(self, xs, ys, figsize:Tuple[int,int]=(12,6), **kwargs):
        "Show the `xs` and `ys` on a figure of `figsize`. `kwargs` are passed to the show method."
        rows = int(math.sqrt(len(xs)))
        fig, axs = plt.subplots(rows,rows,figsize=figsize)
        for i, ax in enumerate(axs.flatten() if rows > 1 else [axs]):
            xs[i].to_one().show(ax=ax, **kwargs)
        plt.tight_layout()

    def show_xyzs(self, xs, ys, zs, figsize:Tuple[int,int]=None, **kwargs):
        """Show `xs` (inputs), `ys` (targets) and `zs` (predictions) on a figure of `figsize`.
        `kwargs` are passed to the show method."""
        figsize = ifnone(figsize, (12,3*len(xs)))
        fig,axs = plt.subplots(len(xs), 2, figsize=figsize)
        fig.suptitle('Ground truth / Predictions', weight='bold', size=14)
        for i,(x,z) in enumerate(zip(xs,zs)):
            x.to_one().show(ax=axs[i,0], **kwargs)
            z.to_one().show(ax=axs[i,1], **kwargs)

In [20]:
a = SideinformationImageImageList.from_csv(path="/mnt/code/repos/DSIN/src/data_paths",csv_name="KITTI_general_test.txt")

In [21]:
a

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/code/repos/DSIN/src/data_paths/data_stereo_flow_multiview/testing/image_2/000122_20.png'

In [10]:
df = pd.read_csv("/mnt/code/repos/DSIN/src/data_paths/KITTI_general_test.txt",header=None, delimiter='\n')


In [11]:
a = df.iloc[0::2]
b = df.iloc[1::2]

In [14]:
a.values[0]

array(['data_stereo_flow_multiview/testing/image_3/000122_19.png'], dtype=object)

In [13]:
class BarakSideinformationImageImageList(ItemList):
    "`ItemList` suitable for `Side Info Image & Image` to `Image` tasks."
    _label_cls,_square_show,_square_show_res = ImageList,False,False
  
    
    def __init__(self, *args, convert_mode:str = 'RGB', after_open:Callable=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.convert_mode,self.after_open = convert_mode,after_open
        self.copy_new += ['convert_mode', 'after_open']
        self.c,self.sizes = 3,{}

    def open(self, fn):
        "Open image in `fn`, subclass and overwrite for custom behavior."
        return open_image(fn, convert_mode=self.convert_mode, after_open=self.after_open)

    def get(self, i):
        fn = super().get(i)
        res = self.open(fn)
        self.sizes[i] = res.size
        return res
    
  
    
#     @classmethod
#     def from_folder(cls, path:PathOrStr='.', extensions:Collection[str]=None, **kwargs)->ItemList:
#         "Get the list of files in `path` that have an image suffix. `recurse` determines if we search subfolders."
#         extensions = ifnone(extensions, image_extensions)
#         return super().from_folder(path=path, extensions=extensions, **kwargs)

   
#     @classmethod
#     def from_csv(cls, path:PathOrStr, csv_name:str, header:str='infer', delimiter:str=None, **kwargs)->'ItemList':
#         "Get the filenames in `path/csv_name` opened with `header`."
#         path = Path(path)
#         df = pd.read_csv(path/csv_name, header=header, delimiter=delimiter)
#         return cls.from_df(df, path=path, **kwargs)

    def reconstruct(self, t:Tensor): return Image(t.float().clamp(min=0,max=1))
    
    
    
    def show_xys(self, xs, ys, imgsize:int=4, figsize:Optional[Tuple[int,int]]=None, **kwargs):
        "Show the `xs` (inputs) and `ys`(targets)  on a figure of `figsize`."
        axs = subplots(len(xs), 3, imgsize=imgsize, figsize=figsize)
        for i, (x,y) in enumerate(zip(xs,ys)):
            x[0].show(ax=axs[i,0], **kwargs)
            x[1].show(ax=axs[i,1], **kwargs)
            y.show(ax=axs[i,2], **kwargs)
        plt.tight_layout()

    def show_xyzs(self, xs, ys, zs, imgsize:int=4, figsize:Optional[Tuple[int,int]]=None, **kwargs):
        "Show `xs` (inputs), `ys` (targets) and `zs` (predictions) on a figure of `figsize`."
        title = 'Input / Prediction / Target'
        axs = subplots(len(xs), 4, imgsize=imgsize, figsize=figsize, title=title, weight='bold', size=14)
        for i,(x,y,z) in enumerate(zip(xs,ys,zs)):
            x[0].show(ax=axs[i,0], **kwargs)
            x[1].show(ax=axs[i,1], **kwargs)
            y.show(ax=axs[i,3], **kwargs)
            z.show(ax=axs[i,2], **kwargs)

In [33]:
x = readfiles1("/mnt/code/repos/DSIN/src/data_paths/KITTI_general_test.txt")

In [42]:
x[0:5][::2]

['data_stereo_flow_multiview/testing/image_3/000122_19.png',
 'data_scene_flow_multiview/testing/image_3/000085_11.png',
 'data_stereo_flow_multiview/testing/image_3/000166_05.png']

In [2]:
size = 32
batchsize = 32
tfms = get_transforms(do_flip = True)
src = (ImageImageList.from_folder(image_path).label_from_func(lambda x: x))
data = (src.transform(tfms, size=size, tfm_y=True)
 .databunch(bs=batchsize)
 .normalize(imagenet_stats, do_y = False))

NameError: name 'image_path' is not defined